<a href="https://colab.research.google.com/github/CaQtiml/Kaggle_Practice/blob/main/Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"caqtiml","key":"931c676d1eabd46e17aa50603090e943"}'}

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-08-03 03:07:10--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-03 03:07:10 (59.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-08-03 03:07:32--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 2607:f8b0:400e:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2021-08-03 03:07:32 (80.6 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [ ]:
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", # watch the val loss metric
                                                  patience=3,
                                                  verbose=1) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7,
                                                 mode='max')

In [ ]:
max_vocab_length = 10000
# max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) 
max_length = 15

In [ ]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length) # how long is each input

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42) # sample 100% of random rows. -> Just Shuffle
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                              train_df_shuffled["target"].to_numpy(),
                                                                              test_size=0.2,
                                                                              random_state=42)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.91%


In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1,activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [ ]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[reduce_lr,early_stopping])

Epoch 1/10
191/191 [==============================] - 8s 17ms/step - loss: 0.5163 - accuracy: 0.7396 - val_loss: 0.4558 - val_accuracy: 0.7951
Epoch 2/10
191/191 [==============================] - 3s 15ms/step - loss: 0.3077 - accuracy: 0.8734 - val_loss: 0.4700 - val_accuracy: 0.7945
Epoch 3/10
191/191 [==============================] - 3s 16ms/step - loss: 0.2048 - accuracy: 0.9233 - val_loss: 0.5976 - val_accuracy: 0.7715

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 4/10
191/191 [==============================] - 3s 15ms/step - loss: 0.1109 - accuracy: 0.9609 - val_loss: 0.6520 - val_accuracy: 0.7748
Epoch 00004: early stopping


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.47866053841102,
 'f1': 0.7737770496754685,
 'precision': 0.7740256742507796,
 'recall': 0.7747866053841103}

In [ ]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=True # keep the pretrained weights (we'll create a feature extractor)
                            )

In [ ]:
model = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1,activation="sigmoid")
    ])

In [ ]:
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model_history = model.fit(train_sentences,
                            train_labels,
                            epochs=10,
                            validation_data=(val_sentences, val_labels),
                            callbacks=[early_stopping,reduce_lr])

Epoch 1/10
191/191 [==============================] - 21s 85ms/step - loss: 0.5923 - accuracy: 0.7365 - val_loss: 0.5034 - val_accuracy: 0.7774
Epoch 2/10
191/191 [==============================] - 16s 82ms/step - loss: 0.4429 - accuracy: 0.8123 - val_loss: 0.4423 - val_accuracy: 0.8043
Epoch 3/10
191/191 [==============================] - 16s 82ms/step - loss: 0.3677 - accuracy: 0.8486 - val_loss: 0.4292 - val_accuracy: 0.8181
Epoch 4/10
191/191 [==============================] - 16s 82ms/step - loss: 0.3083 - accuracy: 0.8767 - val_loss: 0.4293 - val_accuracy: 0.8070
Epoch 5/10
191/191 [==============================] - 16s 83ms/step - loss: 0.2510 - accuracy: 0.9010 - val_loss: 0.4343 - val_accuracy: 0.8168

Epoch 00005: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
Epoch 6/10
191/191 [==============================] - 16s 83ms/step - loss: 0.1944 - accuracy: 0.9300 - val_loss: 0.4441 - val_accuracy: 0.8162
Epoch 00006: early stopping


In [ ]:
model_pred_probs = model.predict(val_sentences)
model_preds = tf.squeeze(tf.round(model_pred_probs))
model_results = calculate_results(val_labels, model_preds)
model_results

{'accuracy': 81.61523309258043,
 'f1': 0.815350947147936,
 'precision': 0.8158180311380308,
 'recall': 0.8161523309258043}

In [ ]:
baseline_pred_probs = np.max(model_0.predict_proba(val_sentences), axis=1) # get the prediction probabilities from baseline model
one = np.round(baseline_pred_probs)
two = np.round(np.squeeze(model_2_pred_probs, axis=1))
three = np.round(np.squeeze(model_pred_probs))
print(one)
print(two)
print(three)

combined = one+two+three
combined_preds = [1 if i>1 else 0 for i in combined]
print(len(combined))
print(len(combined_preds))
# combined_preds = tf.reduce_max(test,axis=0)

# combined_pred_probs = baseline_pred_probs + tf.squeeze(model_2_pred_probs, axis=1) + tf.squeeze(model_pred_probs)
# combined_preds = tf.round(combined_pred_probs/3) # average and round the prediction probabilities to get prediction classes
# print(combined_preds)


[1. 1. 1. ... 1. 1. 1.]
[0. 1. 1. ... 1. 1. 1.]
[0. 1. 1. ... 0. 1. 1.]
1523
1523


In [ ]:
ensemble_results = calculate_results(val_labels, combined_preds)
ensemble_results

{'accuracy': 77.41300065659883,
 'f1': 0.7750340720297387,
 'precision': 0.7810994461325508,
 'recall': 0.7741300065659882}

In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
test_sentences = test_df["text"]
pred_probs = model.predict(test_sentences)
preds = np.squeeze(np.round(pred_probs)).astype(int)

for i,text in enumerate(test_sentences[:20]):
    print(preds[i],pred_probs[i])
    print(text)
    print("-------------------")

ids = test_df["id"]
print(preds.shape)

1 [0.9053067]
Just happened a terrible car crash
-------------------
1 [0.9781033]
Heard about #earthquake is different cities, stay safe everyone.
-------------------
1 [0.9654698]
there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
-------------------
1 [0.9560832]
Apocalypse lighting. #Spokane #wildfires
-------------------
1 [0.9752099]
Typhoon Soudelor kills 28 in China and Taiwan
-------------------
1 [0.91626406]
We're shaking...It's an earthquake
-------------------
0 [0.08600182]
They'd probably still show more life than Arsenal did yesterday, eh? EH?
-------------------
0 [0.03702617]
Hey! How are you?
-------------------
0 [0.03701863]
What a nice hat?
-------------------
0 [0.0411622]
Fuck off!
-------------------
0 [0.05312265]
No I don't like cold!
-------------------
0 [0.04174406]
NOOOOOOOOO! Don't do that!
-------------------
0 [0.03708176]
No don't tell me that!
-------------------
0 [0.05392294]
What if?!
-----------------

In [ ]:
submit = pd.DataFrame({"id":ids,
                       "target":preds})
submit

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
submit.to_csv('twitter.csv',index = False)

In [ ]:
! mkdir /.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c nlp-getting-started -f twitter.csv -m "Test"

100% 22.2k/22.2k [00:02<00:00, 10.7kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets